In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
from mpl_toolkits.mplot3d import Axes3D
import subprocess

In [ ]:
sns.set_color_codes()
sns.set_style('ticks')
sns.set_context('paper')

In [ ]:
enconv = 1.789565e+47  # J per msol.ly^2.a^-2
colwidth = 246 / 72.

In [ ]:
npts = 500
m_mean = 10
m_std = 1
rad0 = 20
dt = 100000
num_iters = 2000
proc = [str(x) for x in ['./cluster', npts, m_mean, m_std, rad0, dt, num_iters]]

In [ ]:
# subprocess.run(proc, cwd='../build/release', stdout=subprocess.PIPE)

In [ ]:
with h5py.File('../build/release/result.h5') as f:
    data = f['dataset'][:]
    
radii = np.sqrt(np.sum(data[:, :3]**2, axis=1))
kine = data[:, 7]
pot = data[:, 8]

times = np.arange(num_iters) * dt

In [ ]:
bound = (kine + pot).max(axis=0) < 0

In [ ]:
plt.figure(figsize=(colwidth, 0.7*colwidth))

ublines = plt.plot(times / 1e6, radii[:, ~bound], 'r', linewidth=1, alpha=0.4)
bdlines = plt.plot(times / 1e6, radii[:, bound], 'b', linewidth=1, alpha=0.4)
plt.ylim(0, 50)
plt.xlim(0, 50)
plt.xlabel('Time [Ma]')
plt.ylabel('Radial position [ly]')
sns.despine()
plt.legend((bdlines[0], ublines[0]), ('Bound', 'Unbound'), loc=2, frameon=True)
plt.tight_layout()
plt.savefig('rads.pdf')

In [ ]:
kine.shape

In [ ]:
plt.plot(np.diff(kine, axis=0), linewidth=1);

In [ ]:
plt.plot(kine[:, bound], 'b', linewidth=1); 

In [ ]:
plt.plot(pot[:, bound], 'b', linewidth=1);

In [ ]:
fig = plt.figure(figsize=(colwidth, 0.7*colwidth))
totkine = np.sum(kine[:, bound], -1) * enconv
totpot = np.sum(pot[:, bound], -1) * enconv

plt.plot(times / 1e6, totkine, 'b--', label='Kin. En.')
plt.plot(times / 1e6, totpot, 'r-.', label='Pot. En.')
plt.plot(times / 1e6, totkine + totpot, 'k', label='Total En.')
# nbd = bound.nonzero()[0].shape[0]

# plt.plot(times / 1e6, 2* totkine / nbd  + totpot / nbd, 'b')
# plt.grid()
plt.hlines(0, *plt.xlim(), linewidth=1, edgecolor='Grey')
sns.despine()

plt.legend(loc=4)
plt.xlabel('Time [Ma]')
plt.ylabel('Energy [J]')

plt.tight_layout()
plt.savefig('boundEnergy.pdf')

In [ ]:
totkine = np.sum(kine[:, ~bound], -1) * enconv
totpot = np.sum(pot[:, ~bound], -1) * enconv

plt.plot(times / 1e6, totkine, 'b--', label='Kin. En.')
plt.plot(times / 1e6, totpot, 'r-.', label='Pot. En.')
plt.plot(times / 1e6, totkine + totpot, 'k', label='Total En.')
plt.legend(loc=4)
plt.grid()
sns.despine()
plt.xlabel('Time [Ma]')
plt.ylabel('Energy [J]')

In [ ]:
fig = plt.figure(figsize=(colwidth, 0.7*colwidth))
totkine = np.sum(kine[:, bound], -1) * enconv
totpot = np.sum(pot[:, bound], -1) * enconv

nbd = bound.nonzero()[0].shape[0]

plt.plot(times / 1e6, 2* totkine / nbd  + totpot / nbd, 'b')
plt.grid()
sns.despine()
plt.xlabel('Time [Ma]')
plt.ylabel('Virial equation [J]')

plt.tight_layout()
plt.savefig('virial.pdf')

In [ ]:
i = -1

In [ ]:
i +=1
print(i)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data[i, 0, :], data[i, 1, :], data[i, 2, :])
ax.set_xlim(-30, 30)
ax.set_ylim(-30, 30)
ax.set_zlim(-30, 30)

In [ ]:
data.shape

In [ ]:
radii.shape

In [ ]:
data[:, 6:7, bound].shape

In [ ]:
cmLoc = np.sum(data[:, :3, bound] * data[:, 6:7, bound], axis=-1) / data[0, 6].sum()
cmPos = data[:, :3] - cmLoc[:, :, np.newaxis]
cmRad = np.sqrt(np.sum(cmPos**2, axis=1))

In [ ]:
xbins, xedges, fig = plt.hist(cmRad[-1, bound], bins=np.linspace(0, 500, 140));
binctrs = xedges[:-1] + (xedges[1] - xedges[0]) / 2
vols = 4/3 * np.pi * (xedges[1:]**3 - xedges[:-1]**3)
dens = xbins / vols
plt.plot(binctrs, xbins, 'r.')

In [ ]:
plt.step(binctrs, vols)

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def fitfunc(r, r0, n0):
    return n0 / (1 + (r / r0)**4)

In [ ]:
fparams, fcov = curve_fit(fitfunc, binctrs, dens)
print(fparams)

In [ ]:
fig = plt.figure(figsize=(colwidth, 0.7*colwidth))

# plt.plot(binctrs, dens, 'o')
# plt.fill_between(xedges[:-1], dens, step='post')
plt.bar(xedges[:-1], dens, width=(xedges[1] - xedges[0]), label='Data')
xs = np.linspace(xedges.min(), xedges.max(), 500)
plt.plot(xs, fitfunc(xs, *fparams), 'k', label='Fit')
plt.xlim(0, 60)
sns.despine()
plt.legend()
plt.xlabel('Radial coordinate [ly]')
plt.ylabel('Particle density [ly^-3]')

plt.tight_layout()
plt.savefig('dens.pdf')

In [ ]:
fig = plt.figure(figsize=(colwidth, 2.2*colwidth))
ax = fig.add_subplot(311, projection='3d')
ax.scatter(cmPos[0, 0, :], cmPos[0, 1, :], cmPos[0, 2, :])
ax.set_xlim(-30, 30)
ax.set_ylim(-30, 30)
ax.set_zlim(-30, 30)

ax = fig.add_subplot(312, projection='3d')
ax.scatter(cmPos[50, 0, :], cmPos[50, 1, :], cmPos[50, 2, :])
ax.set_xlim(-30, 30)
ax.set_ylim(-30, 30)
ax.set_zlim(-30, 30)

ax = fig.add_subplot(313, projection='3d')
ax.scatter(cmPos[-1, 0, :], cmPos[-1, 1, :], cmPos[-1, 2, :])
ax.set_xlim(-30, 30)
ax.set_ylim(-30, 30)
ax.set_zlim(-30, 30)

plt.tight_layout()

plt.savefig('3d.pdf')

In [ ]:
(len(cmPos) - 1) * dt

# Animations

In [ ]:
for i in range(0, len(data), 10):
    plt.plot(data[i, 0, :], data[i, 1, :], '.')
    plt.xlim(-30, 30)
    plt.ylim(-30, 30)
    plt.gca().set_aspect(1)
    plt.savefig('images/anim2d_{:04d}.png'.format(i), bbox_inches='tight')
    plt.close()

In [ ]:
subprocess.call('/usr/local/bin/convert -delay 10 -loop 0 images/anim2d_*.png anim2d.gif', shell=True)

In [ ]:
for i in range(0, data.shape[0], 10):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(data[i, 0, :], data[i, 1, :], data[i, 2, :])
    ax.set_xlim(-30, 30)
    ax.set_ylim(-30, 30)
    ax.set_zlim(-30, 30)
    plt.savefig('images/anim{:04d}.png'.format(i), bbox_inches='tight')
    plt.close()

In [ ]:
subprocess.call('/usr/local/bin/convert -delay 10 -loop 0 images/anim????.png anim.gif', shell=True)